In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy

from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *
from rct_data_generator import generate_host_and_mirror


/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025536958/work/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)
/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set-up

In [2]:
n_host_sample = 100 
sigma_error = 0.01
d = 5
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [3]:

X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.std()) * (Y_host-Y_host.mean()) + sigma_error * torch.randn_like(Y_host)


In [4]:
sigma_error = 1
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}
bayesian_regression = BayesianLinearRegression(prior_hyperparameters)
bayesian_regression.set_causal_index(10)

In [5]:
bayesian_regression.fit(X_host,Y_host)

{'posterior_mean': tensor([-0.2916, -0.8471, -0.4645, -0.7058,  0.7675,  0.8116, -1.1813, -1.2299,
          0.0079,  0.0462]),
 'posterior_cov_matrix': tensor([[ 0.1530,  0.0073, -0.0422, -0.1303, -0.0672, -0.0931,  0.0107,  0.0425,
           0.0318, -0.0206],
         [ 0.0073,  0.1801, -0.0399, -0.0149,  0.0419,  0.0055, -0.1169,  0.0147,
           0.0153, -0.0125],
         [-0.0422, -0.0399,  0.1501, -0.0249, -0.1270,  0.0401,  0.0123, -0.0897,
          -0.0291,  0.0314],
         [-0.1303, -0.0149, -0.0249,  0.3208,  0.2766,  0.0260, -0.0017, -0.0370,
          -0.0722, -0.0075],
         [-0.0672,  0.0419, -0.1270,  0.2766,  0.3809, -0.0192, -0.0241,  0.0204,
          -0.0159, -0.0621],
         [-0.0931,  0.0055,  0.0401,  0.0260, -0.0192,  0.2615,  0.0472, -0.0804,
          -0.1730, -0.0785],
         [ 0.0107, -0.1169,  0.0123, -0.0017, -0.0241,  0.0472,  0.4002, -0.0531,
          -0.0574,  0.0466],
         [ 0.0425,  0.0147, -0.0897, -0.0370,  0.0204, -0.0804, -0.0531

In [6]:
# bayesian_regression.closed_form_obs_EIG(X_host),bayesian_regression.closed_form_causal_EIG(X_host)

In [7]:
# bayesian_regression.samples_obs_EIG(X_host,100,10),bayesian_regression.samples_causal_EIG(X_host,100,10)

In [8]:
T_zero = torch.zeros_like(T_host)
T_one = 1 - T_zero

X_host_times_T_zero = (T_zero.unsqueeze(dim=0).T * X_host_no_T)
X_host_times_T_one = (T_one.unsqueeze(dim=0).T * X_host_no_T)

X_host_T_zero = torch.concat([X_host_no_T,X_host_times_T_zero],dim=1)
X_host_T_one = torch.concat([X_host_no_T,X_host_times_T_one],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.norm()) * Y_host + sigma_error * torch.randn_like(Y_host)

In [9]:
# bayesian_regression.closed_form_obs_EIG(X_host_T_zero),bayesian_regression.closed_form_causal_EIG(X_host_T_zero)

In [10]:
# bayesian_regression.samples_obs_EIG(X_host_T_zero,100,500),bayesian_regression.samples_causal_EIG(X_host_T_zero,100,500)

In [11]:
# bayesian_regression.closed_form_obs_EIG(X_host_T_one),bayesian_regression.closed_form_causal_EIG(X_host_T_one)

In [12]:
# bayesian_regression.samples_obs_EIG(X_host_T_one,100,150),bayesian_regression.samples_causal_EIG(X_host_T_one,100,150)

## Experiment

In [13]:
n_host_sample = 200 
sigma_error = 0
d = 10 
n_repeats = 40


## BART

In [14]:
from outcome_models import BayesianCausalForest
from tqdm import tqdm

In [ ]:
X_host_np = np.array(X_host,dtype=np.float32)
T_host_np = np.array(T_host,dtype=np.int32)
Y_host_np = np.array(Y_host,dtype=np.float32)

In [ ]:
prior_hyperparameters = {'sigma_0_sq':1, 'p_categorical_pr':0, 'p_categorical_trt':0 }
bcf = BayesianCausalForest(prior_hyperparameters,predictive_model_parameters={"num_trees_pr":200,"num_trees_trt":100},conditional_model_param={"num_trees_pr":200})
bcf.store_train_data(X=X_host_np,T=T_host_np,Y=Y_host_np)
bcf.joint_EIG_calc(X_host_np[:50],T_host_np[:50],
                   n_samples_inner_expectation_obs=100,n_samples_outer_expectation_obs=50,
                   n_samples_inner_expectation_caus=100,n_samples_outer_expectation_caus=50)

In [17]:
bcf.store_train_data(X=X_host_np,T=T_host_np,Y=Y_host_np)

In [18]:
n_samples_inner_expectation=10
n_samples_outer_expectation=10
X = X_host_np
T = T_host_np

n_samples = n_samples_outer_expectation*(n_samples_inner_expectation+1)


In [19]:
predicitions = bcf.posterior_sample_predictions(X=X, T=T,  n_samples=n_samples)

In [20]:
from eig_comp_utils import predictions_in_EIG_obs_form,compute_EIG_obs_from_samples

In [21]:
# pred_in_form = predictions_in_EIG_obs_form(predicitions,n_outer_expectation=n_samples_outer_expectation,m_inner_expectation=n_samples_inner_expectation)

In [22]:
# compute_EIG_obs_from_samples(pred_in_form,1)

In [23]:
# bcf.samples_obs_EIG(X_host_np,T_host_np,n_samples_inner_expectation=50,n_samples_outer_expectation=50)

In [24]:
bcf.joint_EIG_calc(X_host_np[:50],T_host_np[:50],
                   n_samples_inner_expectation_obs=100,n_samples_outer_expectation_obs=50,
                   n_samples_inner_expectation_caus=100,n_samples_outer_expectation_caus=50)

Sampling from Posterior
Getting conditional samples


  0%|          | 0/50 [00:00<?, ?it/s]/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/xbcausalforest/xbcf_python.py:286: RuntimeWarning: divide by zero encountered in scalar divide
  self.params["tau_trt"] = 0.1 * np.var(y) / self.params["num_trees_trt"]
100%|██████████| 50/50 [00:45<00:00,  1.11it/s]


{'Posterior Predictive Entropy': 72.53898823460844,
 'Obs EIG': 1.59206157437481,
 'Causal EIG': -1.7993240049358121}

In [ ]:
bcf.samples_obs_EIG(X_host_np,T_host_np,n_samples_inner_expectation=50,n_samples_outer_expectation=50)

In [ ]:
predicitions,tau = bcf.posterior_sample_predictions(X=X, T=T,  n_samples=50,return_tau=True)

In [ ]:
bcf.model.tauhats_adjusted

array([[ 0.08247684,  0.85198329,  0.23105522, ...,  0.21843077,
         0.0092008 , -0.01692819],
       [ 0.08159968,  0.71138705,  0.23105522, ...,  0.19254763,
         0.0092008 , -0.00534278],
       [ 0.08361413,  0.85198329,  0.23105522, ...,  0.19941064,
         0.0092008 , -0.0083654 ],
       ...,
       [ 0.11895726,  0.85198329,  0.23105522, ...,  0.21843077,
         0.0092008 , -0.02980164],
       [ 0.11895726,  0.7328    ,  0.2775234 , ...,  0.24791655,
         0.01687149, -0.01442364],
       [ 0.08159968,  0.71138705,  0.21997896, ...,  0.19254763,
         0.0092008 ,  0.00165294]])

In [ ]:
Y = Y_host_np

In [ ]:
causal_sample = []
print("Getting conditional samples")
for i in tqdm(range(len(tau))):
    Y_resid = Y - tau[i]
    conditional_predictions = bcf.posterior_conditional_predictions(X=X,T=T,Y_residuals=Y_resid,n_samples=200)
    causal_sample.append((predicitions[i],conditional_predictions))

Getting conditional samples


 36%|███▌      | 18/50 [00:34<01:01,  1.92s/it]


KeyboardInterrupt: 

In [ ]:
causal_sample = []
print("Getting conditional samples")
for i in tqdm(range(len(tau))):
    Y_resid = Y - tau[i]
    conditional_predictions = bcf.posterior_conditional_predictions(X=X,T=T,Y_residuals=Y_resid,n_samples=200)
    causal_sample.append((predicitions[i],conditional_predictions))

In [ ]:
from eig_comp_utils import compute_EIG_causal_from_samples

In [ ]:
compute_EIG_causal_from_samples(pred_in_form,causal_sample,1)

2.013498975721575

In [ ]:
preds,tau = bcf.posterior_conditional_predictions(X=X,T=T,Y_residuals=Y_resid,n_samples=200,return_tau=True)

/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/xbcausalforest/xbcf_python.py:286: RuntimeWarning: divide by zero encountered in scalar divide
  self.params["tau_trt"] = 0.1 * np.var(y) / self.params["num_trees_trt"]


In [ ]:
preds.shape

(100, 200)

In [ ]:
import xbcausalforest as xbcf

In [ ]:
NUM_TREES_PR = 200 
NUM_TREES_TRT = 80

In [ ]:
cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=200, 
    burnin=100,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=NUM_TREES_TRT,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(Y_host_np)/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(Y_host_np)/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    # standardize y and unstandardize for prediction
         )

In [ ]:
cf.fit(
    x_t=X_host_np,
    x= X_host_np,
    y=Y_host_np,
    z=T_host_np
)

XBCF(num_sweeps = 200, burnin = 100, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 10, mtry_trt = 10, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.0029744889736175533, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 80, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 0.0012393704056739808, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [ ]:
predicitions = cf.predict(X_host_np,X_host_np,return_mean=False,return_muhat=True)

In [ ]:
b = cf.b
b_adj = (b/( np.expand_dims(b[:,1]-b[:,0],axis= 1)))

tau_adj = predicitions[0]* (b_adj.T[T_host_np])
preds = (tau_adj + predicitions[1])

In [ ]:
Y_adj = (np.expand_dims(Y_host_np,axis=1) - tau_adj).T[0]

In [ ]:
cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=200, 
    burnin=100,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=0,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(Y_host_np)/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(Y_host_np)/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    # standardize y and unstandardize for prediction
         )

cf.fit(
    x_t = np.zeros_like(X_host_np),
    x = X_host_np,
    y = Y_adj,
    z = T_host_np
)

/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/xbcausalforest/xbcf_python.py:286: RuntimeWarning: divide by zero encountered in scalar divide
  self.params["tau_trt"] = 0.1 * np.var(y) / self.params["num_trees_trt"]


XBCF(num_sweeps = 200, burnin = 100, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 10, mtry_trt = 10, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.0027849969323278634, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 0, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = inf, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [ ]:
predicitions = cf.predict(np.zeros_like(X_host_np),X_host_np,return_mean=False,return_muhat=True)

In [ ]:
predicitions

(array([[ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.]]),
 array([[ 0.062448  ,  0.36889832, -0.02182688, ...,  0.40745114,
          0.08932317, -0.01795297],
        [ 0.54190005,  0.55311663,  0.31037042, ...,  0.55837775,
          0.84239695,  0.46283443],
        [-1.37596441, -1.23473215, -1.13664206, ..., -1.50755527,
         -1.16834067, -1.10944947],
        ...,
        [-0.91072695, -0.66860846, -1.09486624, ..., -0.77695684,
         -1.15556815, -1.04539682],
        [-0.10020598, -0.42414434, -0.16395756, ..., -0.48657292,
         -0.1416557 ,  0.10174316],
        [ 0.14673005,  0.2849502 ,  0.0815876 , ...,  0.22618989,
         -0.09158059,  0.06720227]]))

In [ ]:
b = cf.b
b_adj = (b/( np.expand_dims(b[:,1]-b[:,0],axis= 1)))

tau_adj = predicitions[0]* (b_adj.T[T_host_np])
preds = (tau_adj + predicitions[1])

In [ ]:
(Y_adj**2).mean()

0.9287824086259364

In [ ]:
((Y_adj-preds.mean(axis=1))**2).mean()

0.015277002089668952

In [ ]:
tau_adj

array([[ 0.,  0., -0., ...,  0., -0.,  0.],
       [-0., -0., -0., ..., -0.,  0.,  0.],
       [ 0.,  0., -0., ...,  0., -0.,  0.],
       ...,
       [ 0.,  0., -0., ...,  0., -0.,  0.],
       [-0., -0., -0., ..., -0.,  0.,  0.],
       [-0., -0., -0., ..., -0.,  0.,  0.]])

In [ ]:
predicitions

(array([[ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.]]),
 array([[ 0.062448  ,  0.36889832, -0.02182688, ...,  0.40745114,
          0.08932317, -0.01795297],
        [ 0.54190005,  0.55311663,  0.31037042, ...,  0.55837775,
          0.84239695,  0.46283443],
        [-1.37596441, -1.23473215, -1.13664206, ..., -1.50755527,
         -1.16834067, -1.10944947],
        ...,
        [-0.91072695, -0.66860846, -1.09486624, ..., -0.77695684,
         -1.15556815, -1.04539682],
        [-0.10020598, -0.42414434, -0.16395756, ..., -0.48657292,
         -0.1416557 ,  0.10174316],
        [ 0.14673005,  0.2849502 ,  0.0815876 , ...,  0.22618989,
         -0.09158059,  0.06720227]]))

In [ ]:
((preds.mean(axis=1)-Y_host_np)**2).mean()

0.024142782849967778

In [ ]:
preds

array([[ 0.062448  ,  0.36889832, -0.02182688, ...,  0.40745114,
         0.08932317, -0.01795297],
       [ 0.54190005,  0.55311663,  0.31037042, ...,  0.55837775,
         0.84239695,  0.46283443],
       [-1.37596441, -1.23473215, -1.13664206, ..., -1.50755527,
        -1.16834067, -1.10944947],
       ...,
       [-0.91072695, -0.66860846, -1.09486624, ..., -0.77695684,
        -1.15556815, -1.04539682],
       [-0.10020598, -0.42414434, -0.16395756, ..., -0.48657292,
        -0.1416557 ,  0.10174316],
       [ 0.14673005,  0.2849502 ,  0.0815876 , ...,  0.22618989,
        -0.09158059,  0.06720227]])

In [ ]:
Y_host_np

array([-1.0027681e-02,  3.3784306e-01, -1.2541382e+00,  3.1256959e-01,
       -4.7415957e-01,  4.5077965e-01,  2.4700131e+00, -6.7570603e-01,
       -1.4678315e+00, -5.0529778e-01, -2.2587560e-01, -5.4501891e-02,
       -1.3739359e+00, -1.4328986e-01, -3.6615300e-01, -2.5677934e-02,
        9.6452802e-01,  5.3417987e-01,  1.0382600e+00,  1.7982579e-03,
        1.0459331e+00, -3.3536810e-01,  1.2266842e+00, -1.4169722e+00,
        5.5360150e-02, -1.7924825e+00, -6.7738706e-01, -1.5795844e+00,
       -7.6324731e-02,  7.2360235e-01, -2.9838771e-01,  1.4375595e+00,
       -1.3860397e-01, -7.7572024e-01,  2.6350304e-01,  4.5717143e-02,
       -2.2918837e+00, -4.2746985e-01, -1.2554047e+00, -3.5815999e-01,
        1.5554879e+00,  1.1510255e+00, -1.6451495e+00, -9.9534285e-01,
       -1.4149719e-01,  2.4940895e-01, -2.8302634e-01, -2.0933665e-01,
        5.3453165e-01, -2.0229363e+00,  2.2036110e-01,  3.7880665e-01,
       -5.5487955e-01,  3.8057946e-02, -5.8195662e-01, -1.9363624e-01,
      

In [ ]:
zero_one = b_adj
zero_one[:,0] = 0
zero_one[:,1] = 1

In [ ]:
zero_one

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [ ]:
(preds * (zero_one.T[T_host_np]))

array([[ 0.062448  ,  0.36889832, -0.02182688, ...,  0.40745114,
         0.08932317, -0.01795297],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.37596441, -1.23473215, -1.13664206, ..., -1.50755527,
        -1.16834067, -1.10944947],
       ...,
       [-0.91072695, -0.66860846, -1.09486624, ..., -0.77695684,
        -1.15556815, -1.04539682],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.        ,  0.        ]])

In [ ]:
T_host_np

array([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0], dtype=int32)

In [ ]:
b_adj.T[T_host_np].shape

(100, 200)

In [ ]:
1/(b[:,1]-b[:,0])

array([   1.        ,    1.79263863,   16.85242175,    1.27232554,
          1.95618683,   -0.81307613,   -2.90397183,   -0.88551136,
          0.81156604,    2.34651251,    0.85757365,   -0.45146257,
          1.57298871,   -1.04484782,   -0.84375621,    0.70354786,
          6.5773465 ,   78.16367847,    1.09559407,    1.04183923,
          0.96606927,    2.46862989,   -6.17447733,    0.45069441,
         -8.30438462,    3.30564216,   -1.75249179,    1.36468276,
          1.63963622,   -3.60870995,   -4.35719005,   -0.89769291,
         55.39759803,   -1.01059817,   -1.72822839,   -1.38167704,
          0.65313602,   -0.96055083,   -0.50261003,   -0.87914368,
         -4.37855778,    1.12313459,   -9.39800577,   -0.74272738,
          0.73814445,   -0.81535814,    3.11883491,   -1.31026382,
         13.39771173,   -4.6314851 ,   -0.93088136,    3.0983337 ,
         -6.36670132,   -2.74815534,    0.65435172,   -8.08609789,
         -1.30175958,    1.6412005 ,    0.48467231,   -1.53750